(Example from the Book p.29 ff)

A lighthouse is somewhere off a piece of straight coastline at a position $\alpha$ (alpha) along the shore and a distance <br>
$\beta$ (beta) out at sea. It emits a series of short highly collimated flashes at random intervals and hence at random azimuths, $\theta$. <br>
These pulses are intercepted on the coast by photo-detectors that record only the fact that a flash has occurred, but not <br>
the angle from which it came. <br>
N flashes have so far been recorded at positions $\{x_k\}$. Where is the lighthouse?’

<img src="pictures01/lightHouseGeometry.jpg">

If we assume a uniform distribution, $ prob(\theta | \{x_k\}, \alpha, \beta, I) = \frac{1}{\pi}$ for $\theta$ between $[- \pi/2 , + \pi/2]$, we have a Cauchy-Distribution for the $x_k$ :

$$
prob(x_k | \alpha, \beta, I) = \frac{\beta}{\pi [\beta^2 + (x_k - \alpha)^2]}
$$

Using Bayes' theorem and assuming independence of the measurements, the log-prob function L is given by:

$$
  L = log(prob(  \alpha, \beta | \{x_k\} , I) = const + \sum^N_{k=1}{(log(\beta) - log(\beta^2 + (x_k - \alpha)^2))}
$$

Our best estimate for $\alpha$ and $\beta$ are the values that maximize the log-prob function L.
We can take the partial derivatives of L for $\alpha$ and $\beta$ and set them equal to zero and solve the equations numerically. Or we use gradient descent to find the minimum of -L, i.e. the maximum of L.
Thus we end up with a non-liniear optimization problem / non-linear root finding problem that we will have to solve numerically:

$$
\frac{\partial L}{\partial \alpha} = \sum^N_{k=1}{\frac{2 (x_k - \alpha)}{\beta^2 + (x_k - \alpha)^2}} = 0 \\


\frac{\partial L}{\partial \beta} = \frac{N}{\beta} - \sum^N_{k=1}{\frac{2 \beta}{\beta^2 + (x_k - \alpha)^2}} = 0
$$

In [1]:
import numpy as np

a = 1       # on shore position of Lighthouse
b = 2       # off shore position of Lighthouse
N = 10      # number of datapoints /measurements to generate

def generate_x_ks(a, b, N=10):
    thetas = np.random.uniform(low= - np.pi * 0.5, high= np.pi * 0.5, size=(1,N))
    x_ks = a + b * np.tan(thetas)
    return x_ks

# test:
generate_x_ks(a, b, N)

array([[ 4.29331231,  2.16376235,  5.67368268,  2.76885175, -0.01804232,
         3.91458902,  1.25565105,  8.31993907, -0.06938472,  0.41978214]])

In [2]:
# function that gives back the partial derivation of L for a (as a function):
def grad_L_a(x_ks):
    def grad(a, b):
        diffs = x_ks - a
        numerator = diffs
        denominator = b**2 + diffs**2
        fractions = numerator/denominator
        return 2 * np.sum(fractions)  
    return grad

# test:
a = 1
b = 2
x_ks = generate_x_ks(a,b)

dLa = grad_L_a(x_ks)
dLa(5, b)

-1.581228290834572

In [3]:
# function that gives back the partial derivation of L for b (as a function):
def grad_L_b(x_ks):
    def grad(a, b):
        N = np.size(x_ks)
        diffs = x_ks - a
        numerator = b
        denominator = b**2 + diffs**2
        fractions = numerator/denominator
        return (N/b) - 2 * np.sum(fractions)  
    return grad

# test:
a = 1
b = 2
x_ks = generate_x_ks(a,b)
dLb = grad_L_b(x_ks)
dLb(a, b)

1.8212757772981432

In [17]:
# implement gradient descent to find the maximum of L, i.e. the minimum of -L:

def gradient_descent(x, y, x_ks, learning_rate=0.01, N_iterations=1000):
    # y is the strarting-point for the off-shore distance - hence allways positive:
    if y < 0 or y==0:
        return
    dL_da = grad_L_a(x_ks)
    dL_db = grad_L_b(x_ks)
    for _ in np.arange(N_iterations):
        x_old = x
        x = x + learning_rate * dL_da(x, y) # plus learningrate * ... is correct in this case, because we find the minimum of -L
        y = y + learning_rate * dL_db(x_old, y)
    return x,y

# test:
a = 11
b = 2
x_ks = generate_x_ks(a, b, 100)
gradient_descent(43, 7, x_ks)

(11.673254727807697, 2.1554659549453996)

### Additonal things to do:
- plot graphs of $prob(\alpha | \{x_k\}, I)$ and $prob(\beta | \{x_k\}, I)$
- add (Gaussian) noise to the problem - either in $\theta$ or in $x_k$
- calculate error-bars for $\alpha$ and $\beta$ and the correlation-matrix
- solve the problem by calculating the zeros of the partial derivatives (i.e. a different algorithm)
- plot a sequence of graphs showing the successive approximation of the estimated location and the real location of the light-house. Add 90% prob-contours to the plot
- make the sequence of plots an animation